In [3]:
import pandas as pd
from sklearn import linear_model,metrics
from scipy.stats import boxcox
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

sns.set()
%matplotlib inline

In [11]:
train_df = pd.read_csv("train_prepared.csv")
test_df = pd.read_csv("test_prepared.csv")

### Logisticやります
やりかたよくわかってない
  
- `train loop`を決める(?)

In [23]:
def train(df):
    df["preds"] = pd.NA

    drop = ["Transported","preds","kfold"]

    for fold in range(5):
        train = df[df["kfold"] != fold]

        y_train = train["Transported"].values
        x_train = train.drop(drop,axis=1).values

        val = df[df["kfold"] == fold]

        y_val = val["Transported"].values
        x_val = val.drop(drop,axis=1).values

        model = linear_model.LogisticRegression(max_iter=1000)
        model.fit(x_train,y_train)

        preds = model.predict(x_val)
        df.loc[val.index,"preds"] = preds

        acc = metrics.accuracy_score(y_val,preds)
        print(f"Fold{fold+1}-Accuracy = {acc:.4f}")
    
    df[drop] = df[drop].astype(int)

    # 全体の正解率計算
    acc = metrics.accuracy_score(df["Transported"].values,df["preds"].values)
    print(f"Overall accuracy:{acc:.4f}")

    return df

In [24]:
# CabinNumとGrpupId以外
train_df_exp1 = train(train_df.copy())

Fold1-Accuracy = 0.7872
Fold2-Accuracy = 0.7867
Fold3-Accuracy = 0.7936
Fold4-Accuracy = 0.7975
Fold5-Accuracy = 0.7969
Overall accuracy:0.7924


### `train`関数
1. `Transported`,`preds`,`kfold`は学習に使わないよ！

``` python
for fold in range(5)
```

2. kfoldの回数である5回回して学習させるよ！

``` python
train = df[df["kfold"] != fold]
```

- 各イテレートにおうじてtrainデータ決める
- `y_train`は到着できたかどうか
- `x_train`は各データ
- `y_val`,`x_val`も一緒

3. modelはロジスティック回帰を`max_iter`1000にするよ！
4. 学習！
5. 予測！
6. dfに代入！！
7. 正解率計算！
8. owari

In [25]:
#1.0が正解，0.0がミスとして見てみたい
wrong = train_df_exp1[train_df_exp1["Transported"] != train_df_exp1["preds"]]

In [28]:
wrong["GroupSize_1"].value_counts()

1.0    1101
0.0     704
Name: GroupSize_1, dtype: int64